# HW4 Multi-Class Classification (Due: Thursday, February 29, 2024, 11:59 PM)

####❗Please submit this notebook (`.ipynb`) with your solutions. The solutions must include the **code**, **explanations** (e.g. *comments*, *figure title, axis labels, and legend*), and the **output of all the cells**. Submitting your solutions without them will lead to `a deduction of points`.

####❗Also, note that you should **cite all the references** you used under each question. Proper referencing is essential for academic integrity, giving credit to original authors, avoiding plagiarism, and providing a traceable path for verification. `Please check the course syllabus for more details about academic integrity`.

####❗Note that you are *not* allowed to use sklearn unless specified by the question.

####❗Please note that the bonus points will be applied to your overall percentage of the course.
---

#### **Q0)** [[0 point]](https://) While you are allowed to discuss homework assignments, it is essential that you write down your code and solutions yourself. If you have discussed the homework with other students, please mention their names.


\##Your answer here

####Please run the cell below to import libraries needed for this HW. Please use the autograd numpy, otherwise you will have issues. Please remember to always use the np library for mathematical functions (e.g., np.log, np.exp, np.sum, etc)

In [1]:
import autograd.numpy as np
from autograd import grad
import matplotlib.pyplot as plt

#### Please load the MNIST digits classification dataset by running the code below. There are 70,000 samples in the dataset where each sample is represented by a 784-dimensional feature vector. (Reference: https://www.openml.org/d/554)




In [2]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
print(X)
print(y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['5' '0' '4' ... '4' '5' '6']


#### Run the code below to divide the data into train/test sets.

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=10000, test_size=2000)

# $k$ Nearest Neighbors ($k$-NN) Classifier



#### **Q1**) [[30 points]](https://) Implement a $k$-Nearest Neighbors classifier *from scratch*. You are free to code this up in your preferred style. However, make sure that your implementation includes a function that takes an array of feature vectors for the training set, the corresponding class labels, an array of feature vectors for the test set, and the number of nearest neighbors, $k$. The function should return an array of predicted labels for the input test data. Make sure you explain in detail about your implmenetaion.

In [4]:
import numpy as np

def knn_classifier(X_train, y_train, X_test, k):
    
    # Function to compute Euclidean distance between two vectors
    def euclidean_distance(x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    # Function to find k-nearest neighbors for a single test sample
    def get_neighbors(x_test):
        
        # Calculate Euclidean distances between the test sample and all training samples
        distances = [euclidean_distance(x_test, x_train) for x_train in X_train]
        
        # Sort distances in ascending order and select the first 'k' (nearest neighbors)
        indices = np.argsort(distances)[:k]
        return indices

    # Function to perform majority voting among neighbors
    def majority_vote(neighbors):

        # Count occurrences of each label in the neighbors
        counts = np.bincount(y_train[neighbors])

        # Return the label with the highest count (majority vote)
        return np.argmax(counts)

    # Predict the class labels for the test set
    y_pred = []
    for x_test in X_test:
        neighbors = get_neighbors(x_test)
        predicted_label = majority_vote(neighbors)
        y_pred.append(predicted_label)

    # Return array of predicted labels for the input test data
    return np.array(y_pred)


#### **Q2**) [[15 points]](https://) Implement a function to calcluate the accuracy for a multi-class classification model *from scratch*. Given predicted labels and true labels, your function should return the accuracy.

In [5]:
# Function to calculate accuracy
def calculate_accuracy(y_pred, y_true):
    
    # Ensure the lengths of predicted and true labels are the same
    if len(y_pred) != len(y_true):
        raise ValueError("Length mismatch.")

    # Count the number of correctly classified samples
    correct_count = sum(1 for pred, true in zip(y_pred, y_true) if pred == true)

    # Calculate accuracy as the ratio of correctly classified samples to total samples
    accuracy = correct_count / len(y_true)

    return accuracy


#### **Q3**) [[5 points]](https://) Given $k$=3, calculate the accuracy of the test set using the function implemented in Q2.


In [6]:
# Assuming y_train is initially in string format
unique_labels_train = np.unique(y_train)
label_mapping_train = {label: idx for idx, label in enumerate(unique_labels_train)}
y_train_encoded = np.array([label_mapping_train[label] for label in y_train])

# Get predicted labels using the k-NN classifier
k = 3
y_pred = knn_classifier(X_train, y_train_encoded, X_test, k)

# Assuming y_test is initially in string format
unique_labels_test = np.unique(y_test)
label_mapping_test = {label: idx for idx, label in enumerate(unique_labels_test)}

# Check if there are any labels in y_test that were not present in y_train
new_labels = set(y_test) - set(label_mapping_train.keys())
if new_labels:
    print(f"Warning: New labels found in y_test: {new_labels}")

# Convert y_test to encoded labels
y_test_encoded = np.array([label_mapping_train[label] if label in label_mapping_train else label_mapping_test[label] for label in y_test])

# Calculate accuracy using the calculate_accuracy function
accuracy = calculate_accuracy(y_pred, y_test_encoded)

print("Accuracy:", accuracy)


Accuracy: 0.9565


# Naive Bayes Classifier


#### **Q4**) [[60 points]](https://) Implement both a Multinomial Naive Bayes classifier with Laplace smoothing [30 points] and a Gaussian Naive Bayes classifier [30 points] *from scratch*. To prevent everything from becoming zeros, consider adding logs instead of multiplying. You are free to code this up in your preferred style, but you should explain in detail about your implmenetaion.

In [7]:
class MultinomialNaiveBayes:
    
    def __init__(self, smoothing=1.0):
        # Smoothing coefficient for Laplace smoothing
        self.smoothing = smoothing
        # Matrix of feature weights
        self.weights = None
        # Log priors for each class
        self.priors = None

    def fit(self, X, Y):
        # One-hot encoding for Y
        K = len(set(Y)) # Number of classes
        N = len(Y) # Number of samples
        labels = Y
        Y_one_hot = np.zeros((N, K))
        Y_one_hot[np.arange(N), labels] = 1

        # Calculate log-weights and log-priors
        self.weights = np.log(X.T.dot(Y_one_hot) + self.smoothing) - np.log((X.T.dot(Y_one_hot) + self.smoothing).sum(axis=0))
        self.priors = np.log(Y_one_hot.sum(axis=0)) - np.log(Y_one_hot.sum(axis=0).sum())

    def score(self, X, Y):
        # Accuracy of model on given data
        return np.mean(self.predict(X) == Y)

    def predict(self, X):
        # Predicted class labels
        return np.argmax(X.dot(self.weights) + self.priors, axis=1)
    

In [8]:
class GaussianNaiveBayes:
    def __init__(self, epsilon=1e-1):
        # Constructor to initialize the Gaussian Naive Bayes model
        self.class_means = None
        self.class_variances = None
        self.epsilon = epsilon

    def fit(self, X_train, y_train):
        # Fit the model to the training data
        unique_classes = np.unique(y_train)

        # Store mean and variance for each feature in each class
        self.class_means = {}
        self.class_variances = {}

        for cls in unique_classes:
            # Extract features corresponding to the current class
            cls_indices = (y_train == cls)
            cls_features = X_train[cls_indices, :]

            # Calculate mean and variance for each feature
            class_mean = np.mean(cls_features, axis=0)
            class_variance = np.var(cls_features, axis=0) + self.epsilon

            # Store mean and variance for the current class
            self.class_means[cls] = class_mean
            self.class_variances[cls] = class_variance

    def predict(self, X_test):
        # Make predictions for the input data
        predictions = []

        for sample in X_test:
            class_scores = []

            for cls, mean in self.class_means.items():
                # Calculate log-likelihood using Gaussian distribution formula
                log_likelihood = -0.5 * np.sum(np.log(2 * np.pi * self.class_variances[cls]))
                log_likelihood -= 0.5 * np.sum(((sample - mean) ** 2) / (self.class_variances[cls]))
                class_scores.append(log_likelihood)

            # Predict the class with the highest log-likelihood
            predictions.append(np.argmax(class_scores))

        return np.array(predictions)
    

#### **Q5**) [[10 points]](https://) Calculate the accuracy of the test set using both of the Naive Bayes classifiers you implemented in Q4. Use the function you implemented in Q2 to obtain the accuracies.



In [9]:
# Set up
multinomial_nb = MultinomialNaiveBayes()
y_train_int = y_train.astype(int)
multinomial_nb.fit(X_train, y_train_int)

# Predict on the test set
y_pred_multinomial_nb = multinomial_nb.predict(X_test)

# Ensure predictions are integers
y_test_encoded_multinomial = [np.int64(str_value) for str_value in y_test]

# Calculate accuracy for Multinomial Naive Bayes
accuracy_multinomial_nb = calculate_accuracy(y_pred_multinomial_nb, y_test_encoded_multinomial)
print(f"Multinomial Naive Bayes Accuracy:", accuracy_multinomial_nb)

# Instantiate Gaussian Naive Bayes
gaussian_nb = GaussianNaiveBayes()
gaussian_nb.fit(X_train, y_train)

# Predictions for Gaussian Naive Bayes
y_pred_gaussian_nb = gaussian_nb.predict(X_test)

# Ensure predictions are integers
y_test_encoded_gaussian = [np.int64(str_value) for str_value in y_test]

# Calculate accuracy for Gaussian Naive Bayes
accuracy_gaussian_nb = calculate_accuracy(y_pred_gaussian_nb, y_test_encoded_gaussian)
print("Gaussian Naive Bayes Accuracy:", accuracy_gaussian_nb)


Multinomial Naive Bayes Accuracy: 0.841
Gaussian Naive Bayes Accuracy: 0.628


#### **Q6**) [[10 points]](https://) Compare the performance of the three classifiers ($k$-NN, Multinomial Naive Bayes, Gaussian Naive Bayes). Which classifier performs the best? Provide a detailed explanation of your conclusion. Make sure that the comparison is based on the same train/test splits.

The k-NN classifier had an accuracy of 0.9565, the Multinomial Naive Bayes classifier had an accuracy of 0.841 and the Gaussian Naive Bayes classifier had an accuracy of 0.628. The k-NN classifier performs the best, based on the same train/test splits. This could be because the k-NN algorithm is able to capture complex relationships within the data by considering local patterns and nearest neighbors. However, Naive Bayes classifiers make strong independence assumptions. These may not hold in scenarios with intricate dependencies among features. In particular, Gaussian Naives Bayes assumes a Gaussian distribution for each class, and if the data deviates significantly from this assumption, its performance may suffer.


# Support Vector Classifier


#### **Q7**) [[***BONUS*** 0.5%]](https://) Train 10 one-versus-all binary classifiers using the soft-margin SVM cost function and the gradient descent function. You may need to revise your gradient descent function for this task. Randomly initialize all model parameters with a normal distribution having a mean of 0 and a standard deviation of 0.01. Then, train each classifier with `max_its=1000` and `alpha=0.0001`. Consider implementing a sampling strategy to deal with an imbalanced dataset. Set your regularization parameter to 1.0. Save the cost and weight history returned by the gradient descent function. Plot the cost history for the 10 binary classifiers into a single plot.





#### **Q8**) [[***BONUS*** 0.4%]](https://) Implement the one-versus-all multiway classifier using the binary classifiers you implemented above. You are free to code this up in your preferred style. However, make sure that your implementation includes a function that takes an array of feature vectors for the test set and an array of learned model parameters. This classifier should return an array of predicted labels for the input test data.

#### **Q9**) [[***BONUS*** 0.1%]](https://) Calculate the accuracy of the test set using your one-versus-all classifier implemented above. Use the function implemented in Q2.

